# Creating House of PlantAgent

## Housekeeping Imports

In [30]:
import os
from dataclasses import dataclass
from langchain_core.tools import tool
from langgraph.runtime import get_runtime
from langchain.agents import create_agent

# For web-searching tool
from langchain.tools import tool
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper


## Loading API KEY

In [31]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:5])

sk-pr


## System Prompt

In [32]:
SYSTEM_PROMPT = """You are an expert vegan/plant-based nutritionist and meal generator.
Your name is House of PlantAgent.

Rules:
- Think step-by-step.
- Provide a list of ingredients, quantities, and cooking instructions in return to the user query for recipe.
- If the user asks non-recipe related questions, state that your purpose is only to generate user recipes.

You have access to one tool, which you MUST use before generating each new recipe-generated response:

- search_plantbased_recipe: use this to search vegan recipe websites based on the user's stated preferences.
"""

## Defining Runtime Context

In [33]:
@dataclass # What does all this do again?
class RuntimeContext:
    """Custom runtime context schema."""
    pass

## Defining Tool

In [34]:
@tool
def search_plantbased_recipe(query:str) -> str:
    """
    Safe wrapper that always returns a string.
    Use this as the tool passed to the agent.

    Argument 'query': The search query string"""
    runtime = get_runtime(RuntimeContext)
    try:
        wrapper = DuckDuckGoSearchAPIWrapper()
        try:
            # prefer .run()
            result = wrapper.run(query)
        except Exception:
            # fallback if the wrapper method differs
            result = wrapper.invoke(query)

        # If result is already a string, return it.
        if isinstance(result, str):
            return result
        # If it's a dict/list/obj, try to serialise to json as safe fallback
        try:
            return json.dumps(result, default=str, ensure_ascii=False)
        except Exception:
            return str(result)
    except Exception as e:
        # Always return a string without raising so the agent gets a tool response
        return f"ERROR in search_plantbased_recipe: {type(e).__name__}: {e}"




### Testing Tool

In [35]:
out = search_plantbased_recipe.invoke("tofu and noodles recipe")
print(type(out), repr(out)[:200])



<class 'str'> "June 5, 2025 - Easy tofu noodles recipe with pan-fried tofu, crisp veg, and chewy ramen in a sesame-soy sauce. A vegan Japanese stir-fry ready in 20 minutes. Time 20:00 This one pan Asian tofu noodle


## Loading Model

In [36]:
agent = create_agent(
    model = "openai:gpt-5",
    tools=[search_plantbased_recipe],
    system_prompt = SYSTEM_PROMPT,
    context_schema = RuntimeContext,
)

In [37]:
question = "Generate a recipe for tofu at dinner"

for step in agent.stream(
    {"messages": question},
    context=RuntimeContext,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Generate a recipe for tofu at dinner
================================== Ai Message ==================================
Tool Calls:
  search_plantbased_recipe (call_pfpclEVTHhS2kf08YW8cCYQn)
 Call ID: call_pfpclEVTHhS2kf08YW8cCYQn
  Args:
    query: vegan tofu dinner recipe
================================= Tool Message =================================
Name: search_plantbased_recipe

These Vegan Tofu Recipes are simple, healthy and perfect for beginners, too. From lettuce wraps to stir fry, to curry and tacos, easy vegan recipes with tofu will make the family's favorite dinners ! Vegan Sheet Pan Lemon Garlic Tofu Dinner is an all-in-one meal. Flavorful garlicky tofu steaks, with perfectly golden roasted potatoes, tender broccoli, and dairy-free yogurt dill sauce. Find the best vegan tofu recipes for breakfast, lunch & dinner ! Easy, protein-packed plant-based meals perfect for weeknights or meal prep. This

In [38]:
# TODO_1 - replace DuckDuck with a better web search tool
# TODO_2 - return not just text but also URL up to the agent, so the agent can enhance the request.